In [3]:
!pip install datasets

In [4]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

In [5]:
ds = load_dataset("stanfordnlp/imdb")
train = ds["train"]
test = ds["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
results_list = []

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {'precision': precision, 'recall': recall, 'f1': f1}

In [7]:
model_name = "huawei-noah/TinyBERT_General_4L_312D"

training_args = TrainingArguments(
    num_train_epochs=2,
    output_dir="./tinybert_results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=512)

train = train.map(tokenize_function, batched=True)
test = test.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate()

trainer.save_model("./tinybert_model")
tokenizer.save_pretrained("./tinybert_model")

results_list.append({
    "Model": model_name,
    "Precision": results["eval_precision"],
    "Recall": results["eval_recall"],
    "F1 Score": results["eval_f1"]
})

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.319000,0.336879,0.925272,0.783520,0.848516
2,0.245900,0.299232,0.898436,0.877520,0.887855


In [8]:
print(results)

{'eval_loss': 0.2992324233055115, 'eval_precision': 0.8984355803096077, 'eval_recall': 0.87752, 'eval_f1': 0.8878546278683881, 'eval_runtime': 86.482, 'eval_samples_per_second': 289.077, 'eval_steps_per_second': 18.073, 'epoch': 2.0}


In [9]:
tokenizer = AutoTokenizer.from_pretrained("derek-harnett/movie-review-classifier")
model = AutoModelForSequenceClassification.from_pretrained("derek-harnett/movie-review-classifier")

training_args = TrainingArguments(
    output_dir="./distilbert_pretrained_results",
    per_device_eval_batch_size=16,
    evaluation_strategy="no"
)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

test = test.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    eval_dataset=test,
    compute_metrics=compute_metrics
)

results = trainer.evaluate()

results_list.append({
    "Model": "Movie Review Classifier - Pretrained",
    "Precision": results["eval_precision"],
    "Recall": results["eval_recall"],
    "F1 Score": results["eval_f1"]
})

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [10]:
print(results)

{'eval_loss': 0.2742638885974884, 'eval_model_preparation_time': 0.0026, 'eval_precision': 0.9273287996204334, 'eval_recall': 0.93816, 'eval_f1': 0.9327129563350036, 'eval_runtime': 435.2786, 'eval_samples_per_second': 57.434, 'eval_steps_per_second': 3.591}


In [11]:
for result in results_list:
  print(result)

{'Model': 'huawei-noah/TinyBERT_General_4L_312D', 'Precision': 0.8984355803096077, 'Recall': 0.87752, 'F1 Score': 0.8878546278683881}
{'Model': 'Movie Review Classifier - Pretrained', 'Precision': 0.9273287996204334, 'Recall': 0.93816, 'F1 Score': 0.9327129563350036}
